# hpstr:ptrless
Using `hpstr:ptrless` to create tuples that can be loaded by `uproot` and `awkward`.

In [1]:
import uproot
import awkward as ak

In [26]:
f = uproot.open('test.root')

In [33]:
t = f['HPS_Event']
events = ak.zip({
    br: t[br].arrays()
    for br in t.keys(recursive=False)
})

In [41]:
for field in events.fields:
    if all([f.startswith(field) for f in events[field].fields]):
        events[field] = {
            f.strip(field+'.'): events[field][f]
            for f in events[field].fields
        }

In [42]:
events

<Array [{EventHeader: {...}, ...}, ..., {...}] type='196 * {EventHeader: {f...'>

In [6]:
import awkward as ak
import coffea
import coffea.nanoevents

In [13]:
events = coffea.nanoevents.NanoEventsFactory.from_root(
    'test.root', 'HPS_Event',
    schemaclass=coffea.nanoevents.BaseSchema,
    metadata={'dataset':'test'},
).events()

In [30]:
import re
form = {}
for br in events.fields:
    sub_branches = re.split('[/.]', br)
    condensed_sub_branches = [
        b
        for i, b in enumerate(sub_branches)
        if i==0 or b != sub_branches[i-1]
    ]
    location = form.get(condensed_sub_branches[0], dict())
    form[condensed_sub_branches[0]] = location
    for b in condensed_sub_branches[1:-1]:
        sub = location.get(b, dict())
        location[b] = sub
        location = sub
    if condensed_sub_branches[-1] == 'parameters_':
        continue
    location[condensed_sub_branches[-1]] = events[br]

def recurzip(form):
    if isinstance(form, ak.Array):
        return form
    
    if all([isinstance(value, ak.Array) for value in form.values()]):
        print()
        print(form)
        return ak.zip(form)
    
    for field in form:
        form[field] = recurzip(form[field])
    
recurzip(form)


{'fUniqueID': <Array [0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0] type='196 * uint32[parameters...'>, 'fBits': <Array [50331648, 50331648, ..., 50331648] type='196 * uint32[parameters={"...'>}

{'fUniqueID': <Array [0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0] type='196 * uint32[parameters...'>, 'fBits': <Array [50331648, 50331648, ..., 50331648] type='196 * uint32[parameters={"...'>}

{'blocklevel': <Array [2576980377, ..., 2576980377] type='196 * uint32[parameters={"__doc_...'>, 'blocknum': <Array [2576980377, ..., 2576980377] type='196 * uint32[parameters={"__doc_...'>, 'nothing': <Array [2576980377, ..., 2576980377] type='196 * uint32[parameters={"__doc_...'>, 'slotid': <Array [2576980377, ..., 2576980377] type='196 * uint32[parameters={"__doc_...'>, 'type': <Array [2576980377, ..., 2576980377] type='196 * uint32[parameters={"__doc_...'>, 'istype': <Array [True, True, True, ..., True, True] type='196 * bool[parameters={"__...'>}

{'nwords': <Array [2576980377, ..., 2576980377] type='196 * uint

ValueError: cannot broadcast nested list